In [1]:
# Import packages and data
import numpy as np
import pandas as pd

runs = pd.read_csv("Data/runs.csv")
races = pd.read_csv("Data/races.csv")

runSize = runs.shape[0]
raceSize = races.shape[0]

runs["race_id"] = runs["race_id"].astype(int)

In [2]:
def printProgress(i):
    if (i%5000==0):
            print("{}/{} ({:.3f}%)".format(i, runSize, 100*float(i)/float(runSize)))

def addJockeyRecord():
    # Adds a column to runs that shows the overall record of that jockey
    jockeyRecord = np.zeros(runSize)
    
    for i in range(runSize):
        run = runs.iloc[i, :]
        race_id = run["race_id"]
        jockey_id = run["jockey_id"]
        
        prevRuns = runs.loc[(runs["jockey_id"]==jockey_id) & (runs["race_id"]<race_id)]
        
        if (prevRuns.shape[0] > 0):
            sumVal = sum(list(map(int, prevRuns['result'])))
            jockeyRecord[i] = sumVal/float(prevRuns.shape[0])
    
        printProgress(i)
            
    runs["jockey_record"] = jockeyRecord

In [57]:
def addGoingType():
    # Adds a column catagorizing the type of going for this race as wet, fast, or slow
    goingType = []
    
    for i in range(runSize):
        run = runs.iloc[i, :]
        race_id = run["race_id"]
        race = races.loc[races["race_id"]==race_id]
        
        raceGoing = race["going"].values
    
        if (raceGoing == 'GOOD TO FIRM' or raceGoing == 'GOOD' or 
            raceGoing == 'GOOD TO YIELDING' or raceGoing == 'FAST'): #Fast Class
            goingType.append("fast")

        elif (raceGoing == 'YIELDING' or raceGoing == 'YIELDING TO SOFT' 
              or raceGoing == 'SOFT' or raceGoing == 'SLOW'): #Slow Class
            goingType.append("slow")

        else: # Wet Class
            goingType.append("wet")
            
        printProgress(i)
        
    runs["going_type"] = np.array(goingType)
    
def addGoingTypeRecord():
    # Adds a column to runs that shows this horses record on similar turf - uses going_type
    turfTypeRecord = np.zeros(runSize)
    
    for i in range(runSize):
        run = runs.iloc[i, :]
        race_id = run["race_id"]
        horse_id = run["horse_id"]
        going_type = run["going_type"]
        
        horse_runs = runs.loc[runs["horse_id"]==horse_id]
        
        same_turf_prev_runs = horse_runs.loc[(horse_runs["race_id"]<race_id) & (horse_runs["going_type"]==going_type)]
        
        same_turf_num_runs = same_turf_prev_runs.shape[0]
        
        if same_turf_num_runs > 0:
            sumVal = sum(list(same_turf_prev_runs["result"]))
            turfTypeRecord[i] = sumVal/float(same_turf_num_runs)
        
        printProgress(i)
            
    runs["going_type_record"] = turfTypeRecord
    
def oneHotEncodeRunVal(col):
    # Set the NA values to "null" and just have consistant zeros for missing values
    runs[col].fillna("null", inplace=True)
    
    uniqueVals = np.unique(runs[col])
    
    oneHotVals = np.zeros((len(uniqueVals), runSize))
    
    for i in range(runSize):
        printProgress(i)
        
        val = runs.iloc[i,:][col]
        
        if val == 'null':
            continue
            
        for j, uVal in enumerate(uniqueVals):
            if val == uVal:
                
                oneHotVals[j,i] = 1
                break
                
                
    for i, uVal in enumerate(uniqueVals):
        if uVal == 'null':
            continue
        colName = col + "_" + str(uVal)
        runs[colName] = oneHotVals[i]
    
def normalizeColumn(X):
    from sklearn.preprocessing import RobustScaler
    transformer = RobustScaler().fit(X)
    
    return transformer.transform(X)

def addWeightDifference():
    weightDiff = np.zeros(runSize)
    
    for i in range(runSize):
        declared = runs.iloc[i,:]["declared_weight"]
        actual = runs.iloc[i,:]["actual_weight"]
        
        weightDiff[i] = declared - actual
        
        printProgress(i)
        
    weightDiff = normalizeColumn(weightDiff.reshape(-1,1))
    
    runs["weight_difference"] = weightDiff
    
def scaleWeights():
    actualWeightScaled = normalizeColumn(np.array(runs["actual_weight"]).reshape(-1,1))
    declaredWeightScaled = normalizeColumn(np.array(runs["declared_weight"]).reshape(-1,1))
    
    runs["actual_scaled_weight"] = actualWeightScaled
    runs["declared_weight_scaled"] = declaredWeightScaled
    
def addWinPercent():
    
    winPct = np.zeros(runSize)
    
    for i in range(runSize):
        run = runs.iloc[i,:]
        horse_id = run["horse_id"]
        race_id = run["race_id"]
        
        horse_prev_races = runs.loc[(runs["horse_id"]==horse_id) & (runs["race_id"]<race_id)]
        rel_race_num = horse_prev_races.shape[0]
        
        if rel_race_num == 0:
            continue
            
        tempWinPct = sum(horse_prev_races["won"]) / float(rel_race_num)
        winPct[i] = tempWinPct
            
        printProgress(i)
            
    runs["win_percent"] = winPct
    
def addTimeForDistance():
    
    avgTime = np.zeros(runSize)
    
    for i in range(runSize):
        run = runs.iloc[i,:]
        horse_id = run["horse_id"]
        race_id = run["race_id"]
        dist = races.loc[races["race_id"]==race_id]["distance"].values
        
        horse_prev_races = runs.loc[(runs["horse_id"]==horse_id) & (runs["race_id"]<race_id)]
        
        rel_race_num = horse_prev_races.shape[0]
        
        if rel_race_num == 0:
            continue
        
        timeSum = 0
        timeCount = 0
        for j in range(rel_race_num):
            temp_race_id = horse_prev_races.iloc[j,:]["race_id"]
            temp_race = races[races["race_id"]==temp_race_id]

            if temp_race["distance"].values == dist:
                timeSum += horse_prev_races.iloc[j,:]["finish_time"]
                timeCount += 1
        if timeCount != 0:
            avgTime[i] = timeSum/float(timeCount)
        
        printProgress(i)
                
    runs["avg_distance_time"] = avgTime

In [58]:
## POTENTIALS:
# change in odds, start sprint time, avg lengths behind, trainer winningness, 
# change in actual weight since last race, number of races raced (!!)


#addJockeyRecord()
#addGoingType()
#addGoingTypeRecord()
#oneHotEncodeRunVal("horse_type")
#addWeightDifference()
#scaleWeights()
#addWinPercent()
addTimeForDistance()

15000/79447 (18.881%)
20000/79447 (25.174%)
25000/79447 (31.468%)
30000/79447 (37.761%)


KeyboardInterrupt: 

In [33]:
runs.columns

Index(['race_id', 'horse_no', 'horse_id', 'result', 'won', 'lengths_behind',
       'horse_age', 'horse_country', 'horse_type', 'horse_rating',
       'horse_gear', 'declared_weight', 'actual_weight', 'draw',
       'position_sec1', 'position_sec2', 'position_sec3', 'position_sec4',
       'position_sec5', 'position_sec6', 'behind_sec1', 'behind_sec2',
       'behind_sec3', 'behind_sec4', 'behind_sec5', 'behind_sec6', 'time1',
       'time2', 'time3', 'time4', 'time5', 'time6', 'finish_time', 'win_odds',
       'place_odds', 'trainer_id', 'jockey_id', 'horse_type_Brown',
       'horse_type_Colt', 'horse_type_Filly', 'horse_type_Gelding',
       'horse_type_Grey', 'horse_type_Horse', 'horse_type_Mare',
       'horse_type_Rig', 'horse_type_Roan', 'weight_difference',
       'actual_scaled_weight', 'declared_weight_scaled'],
      dtype='object')

In [36]:
runs

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,horse_type_Gelding,horse_type_Grey,horse_type_Horse,horse_type_Mare,horse_type_Rig,horse_type_Roan,weight_difference,actual_scaled_weight,declared_weight_scaled,win_percent
0,0,1,3917,10,0,8.00,3,AUS,Gelding,60,...,1.0,0.0,0.0,0.0,0.0,0.0,-1.095238,1.0,-0.976190,0.000000
1,0,2,2157,8,0,5.75,3,NZ,Gelding,60,...,1.0,0.0,0.0,0.0,0.0,0.0,-1.571429,1.0,-1.452381,0.000000
2,0,3,858,7,0,4.75,3,NZ,Gelding,60,...,1.0,0.0,0.0,0.0,0.0,0.0,-0.345238,0.9,-0.238095,0.076923
3,0,4,1853,9,0,6.25,3,SAF,Gelding,60,...,1.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.4,0.190476,0.076923
4,0,5,2796,6,0,3.75,3,GB,Gelding,60,...,1.0,0.0,0.0,0.0,0.0,0.0,-1.642857,0.8,-1.547619,0.076923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79442,6348,10,1238,6,0,1.25,5,AUS,Gelding,87,...,0.0,0.0,0.0,0.0,0.0,0.0,1.071429,-0.1,1.059524,0.000000
79443,6348,11,985,7,0,2.25,5,NZ,Gelding,84,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333,-0.4,-0.380952,0.000000
79444,6348,12,3147,10,0,3.00,5,NZ,Gelding,83,...,0.0,0.0,0.0,0.0,0.0,0.0,0.583333,-0.3,0.547619,0.000000
79445,6348,13,1982,8,0,2.50,3,AUS,Gelding,82,...,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,-0.4,1.952381,0.000000


7


7